In [1]:
recipes = {}

class Recipe():
    def __init__(self, raw_recipe, complexity=0):
        self.name = list(raw_recipe.keys())[0]
        val = list(raw_recipe.values())[0]
        self.produces = val["Produces"]
        self.time = val["Time"]
        self.requires = val.get("Requires")
        self.consumes = val.get("Consumes")
        self.complexity = complexity
        self.raw = raw_recipe
        self.raw_cost = sum(self.consumes.values()) if self.consumes else 0
        self.cost = self.raw_cost + 10*len(self.requires) if self.requires else 0

    def __repr__(self):
        return "<" + self.name + "> Makes " + str(self.produces) + " in "  + str(self.time) + " time; " + ("Needs: " + str(self.consumes) + "; " if self.consumes else "") + ("Tools: " + ", ".join(self.requires.keys()) + " " if self.requires else "") + "Raw Cost: " + str(self.raw_cost) + " Cost: " + str(self.cost) + " Complexity: " + str(self.complexity)

In [3]:
from json import load
with open('crafting.json') as f:
        Crafting = load(f)

def at(dict, index):
    def get_key(dictionary, n=0):
        if n < 0:
            n += len(dictionary)
        for i, key in enumerate(dictionary.keys()):
            if i == n:
                return key
        raise IndexError
    return (lambda key: {key:dict[key]})(get_key(dict,index))

r = Crafting["Recipes"]
punch = {"punch for wood":r["punch for wood"]}
print(r)

{'craft wooden_pickaxe at bench': {'Produces': {'wooden_pickaxe': 1}, 'Requires': {'bench': True}, 'Consumes': {'plank': 3, 'stick': 2}, 'Time': 1}, 'craft stone_pickaxe at bench': {'Produces': {'stone_pickaxe': 1}, 'Requires': {'bench': True}, 'Consumes': {'cobble': 3, 'stick': 2}, 'Time': 1}, 'wooden_pickaxe for coal': {'Produces': {'coal': 1}, 'Requires': {'wooden_pickaxe': True}, 'Time': 4}, 'iron_pickaxe for ore': {'Produces': {'ore': 1}, 'Requires': {'iron_pickaxe': True}, 'Time': 2}, 'wooden_axe for wood': {'Produces': {'wood': 1}, 'Requires': {'wooden_axe': True}, 'Time': 2}, 'craft plank': {'Produces': {'plank': 4}, 'Consumes': {'wood': 1}, 'Time': 1}, 'craft stick': {'Produces': {'stick': 4}, 'Consumes': {'plank': 2}, 'Time': 1}, 'craft rail at bench': {'Produces': {'rail': 16}, 'Requires': {'bench': True}, 'Consumes': {'ingot': 6, 'stick': 1}, 'Time': 1}, 'craft cart at bench': {'Produces': {'cart': 1}, 'Requires': {'bench': True}, 'Consumes': {'ingot': 5}, 'Time': 1}, 'iron

In [2]:
def get_children(item): # stuff that needs this as a requirement
    return [{i[0]:i[1]} for i in filter(lambda r: (r[1].get("Requires") and item in list(r[1].get("Requires").keys())) or (r[1].get("Consumes") and item in list(r[1].get("Consumes").keys())), r.items())]

In [29]:
R = []

def get_key(d):  # returns key bc python dumb_
    return list(d.keys())[0]

def get_val(d):  # returns val bc python dumb_
    return list(d.values())[0]

def add_recipes(item, layer=0): # recursively adds all recipes, needs R and r declared outside (R:final list, r:beginning list)
    # print(item, layer)
    if(layer>10):
        return
    makes_item = [{i[0]:i[1]} for i in filter(lambda x: item in list(x[1]["Produces"].keys()), r.items())] # find recipes that make •item

    for i in [*makes_item, *get_children(item)]: # for all recipes related to •item
        name = get_key(i)
        # print("Name: " + name)
        requirements = (lambda req, con: [*(list(req.keys()) if req else []), *(list(con.keys()) if con else [])])(get_val(i).get("Requires"), get_val(i).get("Consumes")) # any item that current •item needs as prereq
        # print(requirements)

        recipe_items = [r for l in [list(recipe.produces.keys()) for recipe in R] for r in l]

        if(not any(recipe.name == name for recipe in R) and all(req in recipe_items for req in requirements)): # check if not dupe and every prereq is satisfied
            # print("Added: " + name)
            R.append(Recipe({name:dict(**list(i.values())[0], **{"Complexity":layer})}, layer)) # make new Recipe if new
        if(i not in makes_item): # recurse children
            add_recipes(get_key(get_val(i)["Produces"]), layer+1)

add_recipes("wood")

print(len(R))

25
